In [1]:
from string import punctuation, digits
from nltk import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'+digits
stops = set(stopwords.words('russian'))

# Задание 1. 
Напишите сами любой алгоритм извлечения ключевых слов. Какой угодно, пусть даже интуитивно он будет нелепый, неважно. Конечно, будет интересно придумать что-то хитрое, с опорой на лингвистику (пока ее тут почти что не было). Или свой графовый алгоритм -- можно с networkx, а не на матрицах:) Еще круче, если вы сделаете не только слова, но и фразы (чтобы был шанс выловить "образовательные стандарты" или "леонида юзефовича" целиком). Но можно и воспроизвести что-то из того, что есть выше. Но только не копируйте код, а пишите сами, воспроизводя логику.

In [2]:
path = '/book_dh/1925_Bulgakov_Sobache serdce.txt'

In [3]:
with open(path, 'r', encoding='utf-8') as f:
    text = f.read()

In [187]:
with open('stop_words.txt', 'r', encoding='utf-8') as f:
    stops = f.read()
    stops.split('\n')

In [29]:
def normalize(text):
    words = []
    for word in word_tokenize(text.lower()):
        word = word.strip(punct)
        word = morph.parse(word)[0].normal_form
        if word and word not in stops: 
            words.append(word)
    return words

In [2]:
text = """
За последние сутки в Бельгии от коронавируса скончались 313 человек, общее число умерших с начала распространения вируса составило 5136, говорится в сообщении кризисного центра министерства здравоохранения королевства.
Также за минувшие сутки было диагностировано 1329 новых случаев заражения коронавирусом, общее число заболевших составляет 36138 человек.
При этом, бельгийские власти констатируют снижение числа госпитализированных с коронавирусом, а также пациентов, находящихся в интенсивной терапии.
Всего с начала распространения вируса из больниц был выписан 7961 инфицированный COVID-19.
В минувшую среду власти Бельгии продлили меры социальной изоляции до 3 мая. В начале мая, как ожидается, страна начнет постепенно выходить из "карантина".
Врачи предупреждают, что в ближайшие дни цифры по зараженным, возможно, увеличатся, поскольку несколько дней назад власти организовали проведение тестов на коронавирус в домах престарелых, где их раньше не проводили...
"""

## Rapid Automatic Keyword Extraction

Я решила опробовать алгоритм RAKE на материале русского языка. Алгоритм заключается в том, что текст делится на по стоп словам на последовательности слов. Далее каждой из этих последоватлеьностей присваивается скор, которые равен делению степени встречаемости слов с другими словами на частотность слов. Алгоритм неплохо работает на материале английского языка. Скорее всего на материале русского языка алготим вменяемых результатов показывать не будет, так как в русском языке стоп слова в тексте встрчаются намного реже. Например, в русском языке нет артиклей. Проверим.

In [3]:
import re 
import string
import numpy as np
import itertools

In [4]:
from collections import Counter, defaultdict
from nltk import ngrams

In [5]:
punc = '[?.,!@#$%^&*()_+=-{}/\|":;><"\n]'

In [9]:
word_freq = defaultdict(int)
phrases = []

for sent in re.split(punc, text):
    line = []
    if sent != '':
        for word in word_tokenize(sent.lower()):
            word = word.strip(punct)
            word = morph.parse(word)[0].normal_form
            if word in stops or word.isdigit():
                if line != []:
                    phrases.append(line)
                    line = []
            elif word != '': 
                line.append(word)
                word_freq[word] += 1

In [10]:
words = list(word_freq.keys())
n_col = len(words)

* Матрица встречаемости слов
* Степени слов
* Скор каждого слова
* Скор последовательностей

In [387]:
freq_matrix = np.zeros((n_col, n_col), dtype=float, order='C')

In [388]:
for line in phrases:
    for w in itertools.permutations(line, 2):
        ind_a, ind_b = words.index(w[0]), words.index(w[1])
        freq_matrix[ind_a][ind_b] += 1

In [389]:
freq_matrix

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Степени слов

In [390]:
word_degree = np.sum(freq_matrix, axis=1)

Скор каждого слова

In [391]:
word_score = {word: word_degree[words.index(word)]/word_freq[word] for word in word_freq}

Скор последовательностей

In [6]:
def rake_score(phrases, word_score, top=4):

    phr_score = {}

    for line in phrases: 
        s = [word_score[word] for word in line]
        phr_score[' '.join(line)] = sum(s)
    
    phr_score = sorted(phr_score.items(), key=lambda kv: kv[1], reverse=True)
    #return phr_score[:top]
    return [i[0] for i in phr_score[:top]]

In [393]:
rake_score(phrases, word_score)

[('поскольку несколько день назад власть организовать проведение тест',
  52.83333333333333),
 ('минувший среда власть бельгия продлить мера социальный изоляция',
  48.83333333333333),
 ('бельгийский власть констатировать снижение число госпитализировать',
  28.666666666666664),
 ('страна начать постепенно выходить', 12.0)]

Видно, что алгоритм показывает себя очень плохо для задачи выявления ключевых слов на русском языке. Последовательности слишком длинные, так как не содержат стоп слов. Попробуем добавить в список фраз отдельные слова. Однако неплохо выявляет ключевые мысли.

In [404]:
new_w = []

for word in word_freq:
    ind, freq = words.index(word), word_freq[word]
    freq_matrix[ind_a][ind_b] += ind * freq
    new_w.append([word])

In [406]:
word_degree = np.sum(freq_matrix, axis=1)
word_score = {word: word_degree[words.index(word)]/word_freq[word] for word in word_freq}
rake_score(phrases + new_w, word_score)

[('поскольку несколько день назад власть организовать проведение тест',
  8344.833333333334),
 ('тест', 8299.0),
 ('минувший среда власть бельгия продлить мера социальный изоляция',
  48.83333333333333),
 ('бельгийский власть констатировать снижение число госпитализировать',
  28.666666666666664)]

Удалось вывести в топ слово "тест"

Попробуем делить на кол-во слов

In [7]:
def rake_score2(phrases, word_score, top=4):

    phr_score = {}

    for line in phrases: 
        s = [word_score[word] for word in line]
        phr_score[' '.join(line)] = sum(s)/len(line)
    
    phr_score = sorted(phr_score.items(), key=lambda kv: kv[1], reverse=True)
    #return phr_score[:top]
    return [i[0] for i in phr_score[:top]]

In [409]:
word_degree = np.sum(freq_matrix, axis=1)
word_score = {word: word_degree[words.index(word)]/word_freq[word] for word in word_freq}
rake_score2(phrases + new_w, word_score, top=6)

[('тест', 8299.0),
 ('поскольку несколько день назад власть организовать проведение тест',
  1043.1041666666667),
 ('среда', 7.0),
 ('продлить', 7.0),
 ('мера', 7.0),
 ('социальный', 7.0)]

А что если искуственно делить получившиеся последовательности на несколько меньших?

In [423]:
new_phrases = []

for line in phrases: 
    if len(line) > 3:
        for i in range(2, 4):
            for ng in ngrams(line, i):
                new_phrases.append(ng)
    else: new_phrases.append(line)

In [424]:
freq_matrix = np.zeros((n_col, n_col), dtype=float, order='C')

for line in phrases:
    for w in itertools.permutations(line, 2):
        ind_a, ind_b = words.index(w[0]), words.index(w[1])
        freq_matrix[ind_a][ind_b] += 1

In [425]:
word_degree = np.sum(freq_matrix, axis=1)
word_score = {word: word_degree[words.index(word)]/word_freq[word] for word in word_freq}
rake_score2(new_phrases, word_score, top=6)

[('продлить мера', 7.0),
 ('мера социальный', 7.0),
 ('социальный изоляция', 7.0),
 ('продлить мера социальный', 7.0),
 ('мера социальный изоляция', 7.0),
 ('поскольку несколько', 7.0)]

С добавлением отдельных слов

In [426]:
for word in word_freq:
    ind, freq = words.index(word), word_freq[word]
    freq_matrix[ind_a][ind_b] += ind * freq

word_degree = np.sum(freq_matrix, axis=1)
word_score = {word: word_degree[words.index(word)]/word_freq[word] for word in word_freq}
rake_score(new_phrases + new_w, word_score, top=6)

[('организовать проведение тест', 2785.0),
 ('проведение тест', 2778.0),
 ('тест', 2771.0),
 ('продлить мера социальный', 21.0),
 ('мера социальный изоляция', 21.0),
 ('назад власть организовать', 20.333333333333332)]

In [427]:
rake_score2(new_phrases + new_w, word_score, top=6)

[('тест', 2771.0),
 ('проведение тест', 1389.0),
 ('организовать проведение тест', 928.3333333333334),
 ('продлить мера', 7.0),
 ('мера социальный', 7.0),
 ('социальный изоляция', 7.0)]

Кажется, результаты стали намного лучше

# Задание 2
Возьмите любой собственный набор текстов. Примените алгоритм к текстам, выведите результаты на экран.

In [8]:
def ng_maker(line):
    new_phrases = [ng for i in range(2, 4) for ng in ngrams(line, i)]
    return new_phrases
    
def phrases_maker(text):

    word_freq = defaultdict(int)
    phrases = []

    for sent in re.split(punc, text):
       
        if sent != '':
            pos, line = [], []
            for word in word_tokenize(sent.lower()):
                word = word.strip(punct)
                mrph = morph.parse(word)[0]#.normal_form
                #word = mrph.normal_form
       
                if mrph.tag.POS in ('ADJF', 'NOUN', 'PRTF'):
                    if mrph.tag.POS == 'NOUN': 
                        phrases.append([mrph.normal_form])
                        word_freq[mrph.normal_form] += 1
                    line.append(word)
                    word_freq[word] += 1
                    pos.append(mrph.tag.POS)
                    
                elif len(pos) > 1 and 'NOUN' in set(pos):
            
                    if len(line) > 3:
                        lines = ng_maker(line)
                        phrases += lines
                        line, pos = [], []
                    elif line != []:
                        phrases.append(line)
                        line, pos = [], []
         
    return word_freq, phrases

In [9]:
def freq_martix(phrases, words, word_freq, n_col):
    
    freqm = np.zeros((n_col, n_col), dtype=float, order='C')
    
    for line in phrases:
        if len(line) == 1:
            ind, freq = words.index(line[0]), word_freq[line[0]]
            freqm[ind][ind] += ind * freq
        else:
            for w in itertools.permutations(line, 2):
                ind_a, ind_b = words.index(w[0]), words.index(w[1])
                freqm[ind_a][ind_b] += 1
    return freqm

In [10]:
def rake_keywords(text, top=6):
    word_freq, phrases = phrases_maker(text)
    words = list(word_freq.keys())
    n_col = len(words)
    freq_matrix = freq_martix(phrases, words, word_freq, n_col)   
    word_degree = np.sum(freq_matrix, axis=1)
    word_score = {word: word_degree[words.index(word)]/word_freq[word] for word in word_freq}
    res = rake_score(phrases, word_score, top=top)
    return res

In [97]:
rake_keywords(text, top=16)

['день',
 'май',
 'власть',
 'дом',
 'проведение тестов',
 'тест',
 'проведение',
 'цифра',
 'врач',
 'карантин',
 'страна',
 'изоляция',
 'мера',
 'среда',
 'больница',
 'терапия']

In [11]:
import os

In [98]:
path = 'Мага/book_dh'
count = 6

for file_name in os.listdir(path):
    if file_name != '.DS_Store':
        with open(path + '/' + file_name, 'r', encoding='utf-8') as f:
            text = f.read()
            keys = rake_keywords(text, top=5)
            count -= 1
            if count == 0: break
            print(file_name)
            print(keys)
            print()
            #print('{}: {}'.format(file_name, ' '.join(keys)))

1824_Bulgarin_Pravdopodobnye nebylicy, ili Stranstvovaniya po svetu v dvadcat' devyatom veke.txt
['суда проводник мой', 'мой проводник', 'проводник мой', 'проводник', 'принц телескоп']

1899_Bryusov_Gora Zvezdy.txt
['царевна сеата', 'царица сеата', 'все время царевна', 'царевна знаком надсмотрщика', 'царевна розовые']

1903_Bryusov_Respublika Yuzhnogo Kresta.txt
['дивиль сношение', 'трагедии орас дивиль', 'события орас дивиль', 'дивиль городские хлебопекарни', 'орас дивиль']

1925_Belyaev_Golova professora Douelya.txt
['шауб жертву равино', 'ларь', 'брик', 'дверей доктор равино', 'доктор равино']

1925_Belyaev_Poslednij chelovek iz Atlantidy.txt
['акса-гуам камень', 'время акса-гуам', 'акса-гуам последнего отплывавшего', 'акса-гуам последнего', 'акса-гуам им ту']



# Задание 3
Протестируйте алгоритм на любом документе (документах) из датасета https://github.com/mannefedov/ru_kw_eval_datasets Будет хорошо, если вы напишете свою функцию измерения точности, полноты и F-меры. Желающие могут добавить туде еще расчет кэффициента близости Жаккара.

In [12]:
import json

In [13]:
path = 'ng_1.jsonlines'

In [15]:
ng_1_data = []
with open(path, "r") as read_file:
    for line in read_file:
        ng_1_data.append(json.loads(line)) 

In [16]:
ng_1_data[0].keys()

dict_keys(['keywords', 'title', 'url', 'content', 'summary'])

In [19]:
def jaccard_pos_distance(a, b):
    return 1.0 * len(a&b)/len(a|b)

In [33]:
precisions_freq, recalls_freq, jac_freq = [], [], []
precisions_rake, recalls_rake, jac_rake = [], [], []
 
for text in ng_1_data:
    freq = Counter(normalize(text['content'])).most_common(6)
    freq = set([i[0] for i in freq])
    
    rake = set(rake_keywords(text['content'], top=6))
    real = set(text['keywords'])
    common = len(rake & real)
    
    jac_freq.append(jaccard_pos_distance(rake, real))
    recalls_freq.append(common/len(real)) 
    precisions_freq.append(common/len(rake))
    
    jac_rake.append(jaccard_pos_distance(freq, real))
    recalls_rake.append(common/len(real)) 
    precisions_rake.append(common/len(freq))
    

RAKE

In [35]:
def evaluation(precisions, recalls, jac):
    precision = np.mean(precisions)
    recall = np.mean(recalls)
    jac = np.mean(jac)
    fscore = 2 * ((precision * recall) / (precision + recall))
    return precision, recall, fscore, jac

In [36]:
evaluation(precisions_freq, recalls_freq, jac_freq)

(0.0298582995951417,
 0.02879244715024067,
 0.029315688573286727,
 0.015498502077112584)

FREQ

In [37]:
evaluation(precisions_rake, recalls_rake, jac_rake)

(0.0298582995951417,
 0.02879244715024067,
 0.029315688573286727,
 0.08516762493792143)

Очень все плохо, но на тестовой новосте нормально.